- Anthonny Dayvson Lino Paz
- Questão 2

In [72]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [73]:
class FaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label, folder in enumerate(sorted(os.listdir(root_dir))):
            folder_path = os.path.join(root_dir, folder)
            for img in os.listdir(folder_path):
                self.image_paths.append(os.path.join(folder_path, img))
                self.labels.append(label)

        self.label_to_name = {i: name for i, name in enumerate(sorted(os.listdir(root_dir)))}

    def __len__ (self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

In [74]:
class EmbeddingNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.convnet = nn.Sequential(
            nn.Conv2d(3, 32, 5), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 37 * 37, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )

    def forward(self, x):
        x = self.convnet(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [75]:
def generate_triplets(dataset):
    triplets = []
    class_to_idx = {}
    for idx, (_, label) in enumerate(dataset):
        class_to_idx.setdefault(label, []).append(idx)

    for label, indices in class_to_idx.items():
        if len(indices) < 2:
            continue
        anchor_positive = random.sample(list(zip(indices, indices[1:])), min(len(indices)-1, 2))

        for anchor_idx, positive_idx in anchor_positive:
            negative_label = random.choice([l for l in class_to_idx if l != label])
            negative_idx = random.choice(class_to_idx[negative_label])
            triplets.append((anchor_idx, positive_idx, negative_idx))
    
    return triplets


In [76]:
def treinar_modelo():
    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor()
    ])

    dataset = FaceDataset('../dtLabs/post-processed', transform=transform)
    model = EmbeddingNet()
    criterion = nn.TripletMarginLoss(margin=1.0)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(5):
        triplets = generate_triplets(dataset)
        random.shuffle(triplets)
        total_loss = 0.0

        for anchor_idx, positive_idx, negative_idx in triplets:
            anchor_img, _ = dataset[anchor_idx]
            positive_img, _ = dataset[positive_idx]
            negative_img, _ = dataset[negative_idx]

            anchor = anchor_img.unsqueeze(0)
            positive = positive_img.unsqueeze(0)
            negative = negative_img.unsqueeze(0)

            anchor_emb = model(anchor)
            positive_emb = model(positive)
            negative_emb = model(negative)

            loss = criterion(anchor_emb, positive_emb, negative_emb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")
    return model, dataset

In [77]:
def criar_banco(model, dataset):
    model.eval()
    face_db = {}
    with torch.no_grad():
        for i in range(len(dataset)):
            img, label = dataset[i]
            img = img.unsqueeze(0).cuda()
            emb = model(img).cpu().numpy().flatten()
            name = dataset.label_to_name[label]
            face_db[name] = emb
    return face_db

In [78]:
def get_embedding(model, img_path, transform):
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0).cuda()
    with torch.no_grad():
        emb = model(image).cpu().numpy().flatten()
    return emb

In [79]:
def reconhecer(embedding_consulta, face_db):
    scores = {none: cosine_similarity([embedding_consulta], [emb])[0][0] for nome, emb in face_db.items()}
    nome_identificado = max(scores, key=scores.get)
    return nome_identificado, scores[nome_identificado]

In [ ]:
if __name__ == "__main__":
    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor()
    ])

    model, dataset = treinar_modelo()

    face_db = criar_banco(model, dataset)

    emb_novo = get_embedding(model, '../dtLabs/marcelinho_no_db.jpg', transform)
    face_db['Marcelinho'] = emb_novo

    emb_mascara = get_embedding(model, './dtLabs/marcelinho_na_inferencia.jpg', transform)
    nome, score = reconhecer(emb_mascara, face_db)

    img = Image.open('./dtLabs/marcelinho_na_inferencia.jpg')
    plt.imshow(img)
    plt.title(f"Reconhecido como: {nome}\n Score: {score:.4f}")
    plt.axis("off")
    plt.show()